In [0]:
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install deskew
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
import os
from math import radians, cos, sin, asin, sqrt,pi
from pathlib import Path
from osgeo import gdal

In [0]:
!gdown https://drive.google.com/uc?id=1MjBMO_Ql7kth7VPPX6XkDXW4sr1qtjDV
!gdown https://drive.google.com/uc?id=17Tat38W_M_4yaHS_dHTf2TakNaT-jQel
!unzip StraboDependency.zip
!sh install.sh
!sudo apt-get install python3-dev
!sudo apt-get install software-properties-common
!sudo apt-get install libgeos-dev
!sudo pip3 install -r text-detection-requirements.txt
!sudo add-apt-repository -y ppa:alex-p/tesseract-ocr
!sudo apt-get update
!sudo apt install -y libtesseract-dev libleptonica-dev liblept5
!sudo apt install -y tesseract-ocr
!sudo pip3 install git+https://github.com/spatial-computing/tesserocr
!export LC_ALL=C
%cd strabo-text-recognition-deep-learning/
!wget "https://uob-my.sharepoint.com/:u:/g/personal/rs16175_bristol_ac_uk/ESYILU6Hu4dGoo3gfNr6rHUB75nM6zZjcTvie_3gbY74iA?download=1" --output-document "MapData.zip"
!unzip MapData.zip 

In [0]:
!rm -r test

In [5]:
%cd strabo-text-recognition-deep-learning/

/content/strabo-text-recognition-deep-learning


In [0]:
!unzip text_detection.zip #Rob's OCR base points
# !wget "https://uob-my.sharepoint.com/:u:/g/personal/rs16175_bristol_ac_uk/ESYILU6Hu4dGoo3gfNr6rHUB75nM6zZjcTvie_3gbY74iA?download=1" --output-document "MapData.zip"
# !unzip MapData.zip 

In [0]:
city = "Burnley"
dataBasePath = "Sample/Raw/"+city

def findAllFileIds():
  prjFiles = list(filter(lambda x: x.endswith(".prj"),os.listdir(dataBasePath)))
  return list(map(lambda x: Path(x).stem,prjFiles))

def setup():
  fileIds = findAllFileIds()
  for name in fileIds:
    gdal.Translate(os.getcwd()+"/"+dataBasePath+"/"+name + '.png',
               os.getcwd()+"/"+dataBasePath+"/"+name + '.tif',
               options="-of PNG")
  os.mkdir(city)
# setup()

In [0]:

def allImagesToText():
  maps = findAllFileIds()

  try:
    shutil.rmtree("static/results/test2")
  except:
    pass

  for tile in maps:
    try:
      imageToText('1909064010191')
      retrieveTextWLatLonsFromImage('1909064010191')
    except Exception as e:
      print(e)
      
def imageToText(imageId):
  with open("content/MapTD/data/output/"+city+"/"+imageId+".txt", 'r') as f:
    lines = f.readlines()
  
  os.mkdir(imageId)
  latLonTuples = []
  x = len(lines)
  counter = 0
  for line in lines:
    try:
      ctr, w, h, t = retrieveImageRequiredMetrics(line)
      image = cv2.imread(dataBasePath+"/"+imageId+".png")
      image = subimage(image, center=ctr, theta=t, width=w, height=h)
      image = deskewImage(h,w,image)
      image.save(str(imageId)+"/"+str(counter)+".png")
      lat,lon = mapImagePixelToLatLon(ctr,imageId)
      with open(str(imageId)+"/"+str(counter)+".txt",'a') as f:
        f.write(str(lat)+","+str(lon)+"\n")
    except Exception as e:
      print(e)
      pass
    if counter % 10 == 0:
      print("Line " + str(counter) + " of " + str(x))
    counter += 1
  !zip -r $imageId.zip $imageId
  return latLonTuples

def subimage(image, center, theta, width, height):
   shape = ( image.shape[1], image.shape[0] )
   matrix = cv2.getRotationMatrix2D( center=center, angle=theta, scale=1 )
   image = cv2.warpAffine( src=image, M=matrix, dsize=shape )
   x = int( center[0] - width/2  )
   y = int( center[1] - height/2 )
   image = image[ y-20:y+height+20, x-20:x+width+20 ]
   return image

def retrieveImageRequiredMetrics(textline):
  pointsList = list(map(lambda x: int(float(x)), textline.split(",")[:-2]))
  xs = pointsList[0::2]
  ys = pointsList[1::2]
  coords = np.array(list(zip(xs,ys)))
  (centerX,centerY), (width,height),theta = cv2.minAreaRect(coords)
  return (int(centerX)+1,int(centerY)+1), int(width)+1, int(height)+1, int(theta)

def textFromImage(im):
  return "Hi"
  text = pytesseract.image_to_string(im)
  return text

def deskewImage(h,w,image):
  if h > w:
      image = np.rot90(image)
    
  # grayscale = rgb2gray(image)
  # angle = determine_skew(grayscale)
  # image = rotate(image, angle,resize=True,cval=1)
  image = Image.fromarray(image)
  return image

In [0]:
def mapImagePixelToLatLon(coord,imageId):
  targetFile = dataBasePath+"/"+imageId
  targetFileGeo = targetFile + ".tfw"
  lat,lon = getBaseLatLon()
  Xm, Ym = convertPixelToMetreOffset(coord,targetFileGeo)
  newLatFromBase, newLonFromBase = transformLatLonByXMetres(lat,lon,Ym,Xm)
  return (newLatFromBase, newLonFromBase)

def getBaseLatLon():
  return 53.9153862, -2.6224470139

def convertPixelToMetreOffset(coord,targetFileGeo):
  pixelX, pixelY = coord
  with open(targetFileGeo, 'rb') as f:
    lines = f.readlines()
    lonOffsetFromBase = float(lines[-1].strip())
    latOffsetFromBase = float(lines[-2].strip())
    yPixelsToMetres = 1 / float(lines[-3].strip())
    xPixelsToMetres = 1 / float(lines[0].strip())

    metreOffsetOnY = pixelY / yPixelsToMetres
    metreOffsetOnX = pixelX / xPixelsToMetres

    return (metreOffsetOnX+latOffsetFromBase,metreOffsetOnY+lonOffsetFromBase)
  
def transformLatLonByXMetres(lat,lon,dn,de):
    R=6377563.396 #from their files
    dLat = dn/R
    dLon = de/(R*cos(pi*lat/180))
    latO = lat + dLat * 180/pi
    lonO = lon + dLon * 180/pi

    return (latO,lonO)

In [87]:
import shutil
import json

def getAllTextForCity():
  textTuples = []
  maps = findAllFileIds()
  # shutil.rmtree("static/results/test2")
  exceptedFiles = []
  for tile in maps:
    try:
      textInTile = retrieveTextWLatLonsFromImage('1909064010191')
    except Exception as e:
      print(e)
      exceptedFiles.append(tile)
  return textTuples, exceptedFiles

def retrieveTextWLatLonsFromImage(imageId):
  numFiles = len(os.listdir(imageId))
  for i in range(120,numFiles):
    targetFile = str(imageId)+"/"+str(i)
    targetFileImage = targetFile + ".png"
    !python3 run_command_line.py --image=$targetFileImage
    text = retrieveModelOutputStrabo(imageId)
    with open(str(imageId)+"/"+str(i)+".txt",'r') as f:
      latLons = f.read()[:-1]
    with open("Burnley/"+imageId+".txt",'a') as f:
      f.write(latLons+","+text)
    
    shutil.rmtree("static/results/test2")
  return text

def retrieveModelOutputStrabo(target):
  try:
    basePath = "static/results/test2/"+str(target)+"/"
    topLevelName =  list(filter(lambda x: os.path.isdir(basePath+x),os.listdir(basePath)))[0]
    topLevelName = basePath+topLevelName
    with open(topLevelName+"/final.txt",'rb') as f:
      parsed = (json.load(f))
      strings = " ".join(list(map(lambda x: x['NameAfterDictionary'].replace('\n', ' ').replace('\r', ''),parsed['features'])))
      return strings+"\n"
  except:
    pass

getAllTextForCity()

1909064010191/120.png
defaultdict(<class 'int'>, {'result_path': 'static/results/test2'})
image name is 1909064010191/120.png
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
make: Entering directory '/content/strabo-text-recognition-deep-learning/lanms'
make: 'adaptor.so' is up to date.
make: Leaving directory '/content/strabo-text-recognition-deep-learning/lanms'
resnet_v1_50/block1 (?, ?, ?, 256)
resnet_v1_50/block2 (?, ?, ?, 512)
resnet_v1_50/block3 (?, ?, ?, 1024)
resnet_v1_50/block4 (?, ?, ?, 2048)
Shape of f_0 (?, ?, ?, 2048)
Shape of f_1 (?, ?, ?, 512)
Shape of f_2 (?, ?, ?, 256)
Shape of f_3 (?, ?, ?, 64)
Shape of h_0 (?, ?, ?, 2048), g_0 (?, ?, ?, 2048)
Shape of h_1 (?, ?, ?, 128), g_1 (?, ?, ?, 128)
Shape of h_2 (?, ?, ?, 64), g_2 (?, ?, ?, 64)
Shape of h_3 (?, ?, ?, 32), g_3 (?, ?, ?, 32)
2020-04-24 18:51:04.796031: I t

KeyboardInterrupt: ignored

In [80]:
allImagesToText()

Line 0 of 213
Line 10 of 213
Line 20 of 213
Line 30 of 213
Line 40 of 213
Line 50 of 213
Line 60 of 213
Line 70 of 213
Line 80 of 213
Line 90 of 213
Line 100 of 213
Line 110 of 213
Line 120 of 213
Line 130 of 213
Line 140 of 213
Line 150 of 213
Line 160 of 213
Line 170 of 213
tile cannot extend outside image
Line 180 of 213
Line 190 of 213
tile cannot extend outside image
Line 200 of 213
Line 210 of 213

zip error: Nothing to do! (.zip)
1909064010191/0.png
defaultdict(<class 'int'>, {'result_path': 'static/results/test2'})
image name is 1909064010191/0.png
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
make: Entering directory '/content/strabo-text-recognition-deep-learning/lanms'
make: 'adaptor.so' is up to date.
make: Leaving directory '/content/strabo-text-recognition-deep-learning/lanms'
resnet_v1_50/block1 (?, ?, ?, 256)
res